In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )
CountiesTable      = os.path.join(dir_inputs ,r"counties.csv"             )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )



def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Residential-Capacity


In [3]:
AnalysisAreas

'E:\\GitHub\\Residential-Capacity\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#sdf_AA

sdf_AAid = pd.DataFrame.spatial.from_featureclass(AnalysisAreas_IDs)
sdf_AAid

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE
0,0,599.400903,1.220382e+04,4,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,1554.841774,8.561669e+04,4,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
2,2,254.967270,3.811046e+03,4,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
3,3,2666.392431,2.609549e+05,4,a4,"{""rings"": [[[420869.50710000005, 4482955.5603]..."
4,4,11708.780939,1.126298e+06,4,a4,"{""rings"": [[[421226.04870000016, 4479294.0206]..."
...,...,...,...,...,...,...
11136,11136,2229.618596,2.944859e+05,10,g3,"{""rings"": [[[414942.4210000001, 4550112.225299..."
11137,11137,856.947230,2.329713e+04,10,g3,"{""rings"": [[[417057.5292999996, 4550162.428400..."
11138,11138,966.598419,5.007495e+04,10,g3,"{""rings"": [[[416772.02419999987, 4550546.96120..."
11139,11139,8681.643024,8.610622e+05,10,g3,"{""rings"": [[[414009.90950000007, 4551759.66699..."


In [5]:
##create map centered on Salt Lake
#map_areas = gis.map('Salt Lake')
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##classify land use
#arcade_expression_aa = ("var v = $feature.Analysis_1;"
#                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
#                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
#                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
#                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
#                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
#                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
#                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
#                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
#                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
#                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
#                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
#                        "else                                      { return 'other' ; }")
#
##symbology for enrollment classes
#uv_aa = [  
#          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
#        ]
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_areas,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_areas.layout.height='500px'
#map_areas.legend=True
#
##map title
#display(Markdown('<h2><center>Analysis Areas</center></h2>'))
#
##display map
#map_areas

# Parameters

In [6]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


In [7]:
df_ClassParam.dtypes

ClassID                  object
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA         float64
CapacityCom_FAR         float64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
RedevProb                 int64
SFRedevFullAdd           object
ClassOrder                int64
dtype: object

In [8]:
df_AADevCat = pd.read_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'))
df_AADevCat

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,0.000000,0.000000
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996
...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,0.000000,0.000000
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,0.320457,0.000059
2449,77,g2,1.0,Remain,14.0,5.365716e-06,5.961774,5.687062,15730.287215,0.003396,5.682825,0.004237


# Calculate New Jobs/HH

In [9]:
#join Analysis Areas with class parameters
df_AADevCatwParam = pd.DataFrame.merge(df_AADevCat,df_ClassParam,on='ClassID',how='left')
df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,...,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,...,40000,1000000,AND,50,300,20,300,1,Yes,1
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,...,40000,1000000,AND,50,300,20,300,1,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,...,1500000,1200000,AND,60,120,20,120,1,No,27
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,...,1500000,1200000,AND,60,120,20,120,1,No,31
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,...,1500000,1200000,AND,60,120,20,120,1,No,31
2449,77,g2,1.0,Remain,14.0,5.365716e-06,5.961774,5.687062,15730.287215,0.003396,...,1500000,1200000,AND,60,120,20,120,1,No,31


In [10]:
#remove percent open space from available acreage
df_AADevCatwParam['AcresOpenSpaceRemoved'] = df_AADevCatwParam['Acres'] * (1-df_AADevCatwParam['PercentOpenSpace'])
pd.set_option('display.max_columns', None)
display(df_AADevCatwParam)
display(df_AADevCatwParam.columns)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139118
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27,13.654417
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,85.148791
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.288464
2449,77,g2,1.0,Remain,14.0,5.365716e-06,5.961774,5.687062,15730.287215,0.003396,5.682825,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,5.118356


Index(['AreaID', 'ClassID', 'county_id', 'DevCategory', 'parcel_id',
       'job_spaces', 'resunits', 'Acres', 'sf_res', 'sf_com', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'RedevProb',
       'SFRedevFullAdd', 'ClassOrder', 'AcresOpenSpaceRemoved'],
      dtype='object')

In [11]:
#calculate Com and Res Acreage

#initialize - acres_remain and acres_developed don't have res/com breakdown
df_AADevCatwParam['acres_undevelopable'] = 0.0
df_AADevCatwParam['acres_remain'       ] = 0.0
df_AADevCatwParam['acres_redevelopable'] = 0.0
df_AADevCatwParam['acres_developable'  ] = 0.0

#set acres
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'acres_undevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'acres_remain'       ] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'acres_redevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable')  , 'acres_developable'  ] = df_AADevCatwParam['Acres']


#calculate Com and Res SF


#initialize
df_AADevCatwParam['SFcom_undevelopable'   ] = 0.0
df_AADevCatwParam['SFres_undevelopable'   ] = 0.0

df_AADevCatwParam['SFcom_remain'          ] = 0.0
df_AADevCatwParam['SFres_remain'          ] = 0.0

df_AADevCatwParam['SFcom_redeveloped_orig'] = 0.0
df_AADevCatwParam['SFres_redeveloped_orig'] = 0.0

df_AADevCatwParam['SFcom_redeveloped_new' ] = 0.0
df_AADevCatwParam['SFres_redeveloped_new' ] = 0.0

df_AADevCatwParam['SFcom_developed'       ] = 0.0
df_AADevCatwParam['SFres_developed'       ] = 0.0


#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFcom_undevelopable'     ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFres_undevelopable'     ] = df_AADevCatwParam['sf_res']

#set remaining sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFcom_remain'            ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFres_remain'            ] = df_AADevCatwParam['sf_res']


# SFCom = AcresNoOpenSpace / ((SPLITres/SPLITcom)/(HHperAcre*SFperHH) + (1/(FAR*43560)))
# SFRes = SFCom * SPLITres/SPLITcom
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFcom_redeveloped_orig'  ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFres_redeveloped_orig'  ] = df_AADevCatwParam['sf_res']


# temporarily set a very low number for zero to allow calcs to run

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0), 'SFSplitCom'] = 0.00000000001
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0), 'SFSplitRes'] = 0.00000000001

#Split Commercial and Split Residential > 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['SFcom_redeveloped_new'] * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_developed'         ] = (df_AADevCatwParam['SFcom_developed']       * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0.00000000001), 'SFSplitCom'] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0.00000000001), 'SFSplitRes'] = 0

#Split Commercial = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_redeveloped_new'   ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_developed'         ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

#Split Residential = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_redeveloped_new'   ] = 0

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_developed'         ] = 0

df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139118,0.000000,0.000000,0.000000,234.281316,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142,0.000000,0.000000,84.596637,0.000000,0.000000,0.0,0.000000,0.000000,42.155347,10868.498573,0.0,431442.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27,13.654417,15.171574,0.000000,0.000000,0.000000,2228.310208,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,85.148791,0.000000,0.000000,0.000000,94.609768,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1138220.0,2655846.0
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.288464,0.000000,0.000000,0.320516,0.000000,0.000000,0.0,0.000000,0.000000,0.000002,1186.039267,3856.0,8997.0,0.0,0.0
2449,77,g2,1.0,Remain,14.0,5.365716e-06,5.961774,5.687062,15730.287215,0.003396,5.682825,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,5.118356,0.000000,5.687062,0.000000,0.000000,0.000000,0.0,0.003396,15730.287215,0.000000,0.000000,0.0,0.0,0.0,0.0


In [12]:
df_AADevCatwParam[df_AADevCatwParam['AreaID']==52]

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
1635,52,a1,0.0,Undevelopable,2.0,0.000000,1.182429e-05,0.227345,1.182429e-02,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.193243,0.227345,0.000000,0.000000,0.000000,0.000000,0.011824,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0
1636,52,a1,4.0,Developable,488.0,0.000000,0.000000e+00,335.164695,0.000000e+00,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,284.889991,0.000000,0.000000,0.000000,335.164695,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,1709339.0
1637,52,a1,4.0,Redevelopable,2.0,0.000000,1.476300e-08,0.001606,7.821439e-05,0.000000,0.001606,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.001365,0.000000,0.000000,0.001606,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000078,0.0,8.0,0.0,0.0
1638,52,a1,4.0,Remain,120.0,0.000000,7.333449e+01,174.673693,1.615152e+05,0.000000,174.673693,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,148.472639,0.000000,174.673693,0.000000,0.000000,0.000000,0.000000,0.000000,1.615152e+05,0.0,0.000000,0.0,0.0,0.0,0.0
1639,52,a1,4.0,Undevelopable,6.0,0.000000,8.313927e-01,0.295179,4.056365e+03,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.250902,0.295179,0.000000,0.000000,0.000000,0.000000,4056.365161,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0
1640,52,b2,4.0,Developable,314.0,0.000000,0.000000e+00,247.134640,0.000000e+00,0.000000,0.000000,0.000000,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,210.064444,0.000000,0.000000,0.000000,247.134640,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,1838063.0
1641,52,b2,4.0,Redevelopable,26.0,0.000000,6.560000e+00,29.356624,1.940448e+04,0.000000,29.356624,0.000000,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,24.953131,0.000000,0.000000,29.356624,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,19404.481078,0.0,218339.0,0.0,0.0
1642,52,b2,4.0,Remain,81.0,0.000126,3.354706e+01,43.203251,1.240932e+05,0.123166,43.193701,0.009551,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,36.722764,0.000000,43.203251,0.000000,0.000000,0.000000,0.000000,0.123166,1.240932e+05,0.0,0.000000,0.0,0.0,0.0,0.0
1643,52,b2,4.0,Undevelopable,521.0,0.000000,8.301769e-05,874.767264,2.858299e-01,0.000000,0.000000,0.000000,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,743.552175,874.767264,0.000000,0.000000,0.000000,0.000000,0.285830,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0
1644,52,b3,4.0,Developable,1364.0,0.000000,0.000000e+00,377.821257,0.000000e+00,0.000000,0.000000,0.000000,Single Family B3,1.0,0.0,2.5,0.5,3500,350,0.15,150000,1000000,AND,50,300,20,300,1,Yes,8,321.148068,0.000000,0.000000,0.000000,377.821257,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,2810045.0


In [13]:
df1 = df_AADevCatwParam[df_AADevCatwParam.isna().any(axis=1)]
display(df1)

df2 = df_AADevCatwParam[(df_AADevCatwParam == np.inf).any(axis=1)]
display (df2)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


In [14]:
#calculate jobs and HH

#initialize
df_AADevCatwParam['job_spaces_undevelopable'   ] = 0.0
df_AADevCatwParam['job_spaces_remain'          ] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_orig'] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_new' ] = 0.0
df_AADevCatwParam['job_spaces_developed'       ] = 0.0

df_AADevCatwParam['resunits_undevelopable'     ] = 0.0
df_AADevCatwParam['resunits_remain'            ] = 0.0
df_AADevCatwParam['resunits_redeveloped_orig'  ] = 0.0
df_AADevCatwParam['resunits_redeveloped_new'   ] = 0.0
df_AADevCatwParam['resunits_developed'         ] = 0.0

#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'job_spaces_undevelopable'   ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'job_spaces_remain'          ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_orig'] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_new' ] = (df_AADevCatwParam['SFcom_redeveloped_new'] / df_AADevCatwParam['SFperEmp']).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'job_spaces_developed'       ] = (df_AADevCatwParam['SFcom_developed'      ] / df_AADevCatwParam['SFperEmp']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'resunits_undevelopable'     ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'resunits_remain'            ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_orig'  ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_new'   ] = (df_AADevCatwParam['SFres_redeveloped_new'] / df_AADevCatwParam['SFperHH' ]).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'resunits_developed'         ] = (df_AADevCatwParam['SFres_developed'      ] / df_AADevCatwParam['SFperHH' ]).astype('int64')


df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,19.0
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,199.139118,0.000000,0.000000,0.000000,234.281316,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,298.0
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,71.907142,0.000000,0.000000,84.596637,0.000000,0.000000,0.0,0.000000,0.000000,42.155347,10868.498573,0.0,431442.0,0.0,0.0,0.000000,0.000000,4.323625e-02,0.0,0.0,0.0,0.000000,8.002301,107.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27,13.654417,15.171574,0.000000,0.000000,0.000000,2228.310208,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,5.958049,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,85.148791,0.000000,0.000000,0.000000,94.609768,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1138220.0,2655846.0,0.000000,0.000000,0.000000e+00,0.0,3794.0,0.0,0.000000,0.000000,0.0,1770.0
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31,0.288464,0.000000,0.000000,0.320516,0.000000,0.000000,0.0

In [15]:
df_AADevCatwParam[df_AADevCatwParam['AreaID']==52]

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
1635,52,a1,0.0,Undevelopable,2.0,0.000000,1.182429e-05,0.227345,1.182429e-02,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.193243,0.227345,0.000000,0.000000,0.000000,0.000000,0.011824,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000012,0.000000,0.000000e+00,0.0,0.0
1636,52,a1,4.0,Developable,488.0,0.000000,0.000000e+00,335.164695,0.000000e+00,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,284.889991,0.000000,0.000000,0.000000,335.164695,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,1709339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,427.0
1637,52,a1,4.0,Redevelopable,2.0,0.000000,1.476300e-08,0.001606,7.821439e-05,0.000000,0.001606,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.001365,0.000000,0.000000,0.001606,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000078,0.0,8.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.476300e-08,0.0,0.0
1638,52,a1,4.0,Remain,120.0,0.000000,7.333449e+01,174.673693,1.615152e+05,0.000000,174.673693,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,148.472639,0.000000,174.673693,0.000000,0.000000,0.000000,0.000000,0.000000,1.615152e+05,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,73.334487,0.000000e+00,0.0,0.0
1639,52,a1,4.0,Undevelopable,6.0,0.000000,8.313927e-01,0.295179,4.056365e+03,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,Yes,1,0.250902,0.295179,0.000000,0.000000,0.000000,0.000000,4056.365161,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.831393,0.000000,0.000000e+00,0.0,0.0
1640,52,b2,4.0,Developable,314.0,0.000000,0.000000e+00,247.134640,0.000000e+00,0.000000,0.000000,0.000000,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,210.064444,0.000000,0.000000,0.000000,247.134640,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,1838063.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,525.0
1641,52,b2,4.0,Redevelopable,26.0,0.000000,6.560000e+00,29.356624,1.940448e+04,0.000000,29.356624,0.000000,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,24.953131,0.000000,0.000000,29.356624,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,19404.481078,0.0,218339.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,6.560000e+00,62.0,0.0
1642,52,b2,4.0,Remain,81.0,0.000126,3.354706e+01,43.203251,1.240932e+05,0.123166,43.193701,0.009551,Single Family B2,1.0,0.0,2.5,0.5,3500,350,0.15,125000,1000000,AND,50,300,20,300,1,Yes,7,36.722764,0.000000,43.203251,0.000000,0.000000,0.000000,0.000000,0.123166,1.240932e+05,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000126,0.0,0.0,0.0,0.000000,33.547060,0.000000e+00,0.0,0.0
1643,52,b2,4.0,Undevelopable,521.0,0.000000,8.301769e-05,874.76

In [16]:
#calculate totals

df_AASummary = df_AADevCatwParam.groupby(['county_id','AreaID','ClassID','ClassDescription','ClassOrder'], as_index=False).agg({"DevCategory":[np.size], "parcel_id":[np.sum], "Acres":[np.sum], "acres_undevelopable":[np.sum], "acres_remain":[np.sum], "acres_redevelopable":[np.sum], "acres_developable":[np.sum], "SFcom_undevelopable":[np.sum], "SFres_undevelopable":[np.sum], "SFcom_remain":[np.sum], "SFres_remain":[np.sum], "SFcom_redeveloped_orig":[np.sum], "SFres_redeveloped_orig":[np.sum], "SFcom_redeveloped_new":[np.sum], "SFres_redeveloped_new":[np.sum], "SFcom_developed":[np.sum], "SFres_developed":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})
df_AASummary.columns = df_AASummary.columns.droplevel(1)

df_AASummaryTotals = df_AASummary.copy()

df_AASummaryTotals['job_spaces_orig'  ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_orig']).astype('int64')
df_AASummaryTotals['job_spaces_new'   ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_new' ] + df_AASummaryTotals['job_spaces_developed']).astype('int64')
df_AASummaryTotals['job_spaces_change'] = (df_AASummaryTotals['job_spaces_new'          ] - df_AASummaryTotals['job_spaces_orig'  ]).round(0)

df_AASummaryTotals['resunits_orig'    ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_orig'  ]).astype('int64')
df_AASummaryTotals['resunits_new'     ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_new'   ] + df_AASummaryTotals['resunits_developed'  ]).astype('int64')
df_AASummaryTotals['resunits_change'  ] = (df_AASummaryTotals['resunits_new'            ] - df_AASummaryTotals['resunits_orig'    ]).round(0)

df_AASummaryTotals = df_AASummaryTotals[['county_id','AreaID','ClassID','ClassDescription','ClassOrder','parcel_id','Acres','acres_redevelopable','acres_developable','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','job_spaces_undevelopable','job_spaces_remain','job_spaces_redeveloped_orig','job_spaces_redeveloped_new','job_spaces_developed','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

display(df_AASummaryTotals.sum())

df_AASummaryTotals['density_hhemp_orig'  ] = ((df_AASummaryTotals['job_spaces_orig'  ] + df_AASummaryTotals['resunits_orig'     ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_new'   ] = ((df_AASummaryTotals['job_spaces_new'   ] + df_AASummaryTotals['resunits_new'      ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_change'] =  (df_AASummaryTotals['density_hhemp_new'] - df_AASummaryTotals['density_hhemp_orig']).round(1)


county_id                                                                   1842
AreaID                                                                     28391
ClassID                        a1a3a4a5c1a2a4b4b4a1a4d3a1a5b4b5b1c1d1b3a1c3a2...
ClassDescription               Single Family A1Single Family A3Single Family ...
ClassOrder                                                                 11570
parcel_id                                                                 939717
Acres                                                                     316819
acres_redevelopable                                                      15694.1
acres_developable                                                        96058.9
job_spaces_orig                                                           325700
job_spaces_new                                                           1899228
job_spaces_change                                                        1573528
resunits_orig               

In [17]:
df_AASummaryTotals
#export to csv
df_AASummaryTotals.to_csv(os.path.join(dir_results, r'SummaryTable_AreaClass_wTotals.csv'))

In [18]:
##create map centered on Salt Lake
#map_parcels_areas_den = gis.map('Salt Lake')
#
#sdf_AADevCatwParam = pd.DataFrame.merge(sdf_AAid, df_AASummaryTotals, on=['AreaID','ClassID'], how='left')
#
##expression to classify employment growth
#arcade_expression_denchange = ("var v = $feature.density_hhemp_change;"
#                                     "if      (v<0    ) { return 'class0'; }"
#                                     "if      (v<10   ) { return 'class1'; }"
#                                     "else if (v<25   ) { return 'class2'; }"
#                                     "else if (v<50   ) { return 'class3'; }"
#                                     "else              { return 'class4'; }")
#
##symbology for enrollment classes
#uv_denchange = [
#                #{"value":"class0", "label":"Density Decline", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class1", "label":"0 to 10"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class2", "label":"10 to 25"     , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class3", "label":"25 to 50"     , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                {"value":"class4", "label":"More than 50" , "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
#               ]
#
#
##define sdf layer
#sdf_AADevCatwParam.spatial.plot(map_widget = map_parcels_areas_den,
#                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                unique_values=uv_denchange,
#                                arcade_expression=arcade_expression_denchange,
#                                default_symbol="" #don't include an 'other' category
#                                )
#
##define map characteristics
#map_parcels_areas_den.layout.height='500px'
#map_parcels_areas_den.legend=True
#
##map title
#display(Markdown('<h2><center>Density Change by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_den

# Area Summary

In [19]:
df_AreaSummary = df_AASummaryTotals.groupby(['AreaID'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum],"resunits_undevelopable":[np.sum],"resunits_remain":[np.sum],"resunits_redeveloped_orig":[np.sum],"resunits_redeveloped_new":[np.sum],"resunits_developed":[np.sum]})
df_AreaSummary.columns = df_AreaSummary.columns.droplevel(1)

#read in csv with area names
df_AreaNames = pd.read_csv(AnalysisAreasTable)

#merge area names with area summary
df_AreaSummary_wNames = pd.DataFrame.merge(df_AreaNames, df_AreaSummary, on="AreaID")

#add total line to area summary with names
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames.append(df_AreaSummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_AreaSummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_AreaSummary_wNames_wTotals['job_spaces_orig'  ] + df_AreaSummary_wNames_wTotals['resunits_orig'     ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_new'   ] = ((df_AreaSummary_wNames_wTotals['job_spaces_new'   ] + df_AreaSummary_wNames_wTotals['resunits_new'      ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_change'] =  (df_AreaSummary_wNames_wTotals['density_hhemp_new'] - df_AreaSummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_AreaSummary_wNames_wTotals['job_spaces_orig'     ] = (df_AreaSummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_new'      ] = (df_AreaSummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_change'   ] =  df_AreaSummary_wNames_wTotals['job_spaces_new'    ] - df_AreaSummary_wNames_wTotals['job_spaces_orig'] 

df_AreaSummary_wNames_wTotals['resunits_orig'       ] = (df_AreaSummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_new'        ] = (df_AreaSummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_change'     ] =  df_AreaSummary_wNames_wTotals['resunits_new'      ] - df_AreaSummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames_wTotals.set_index(['AreaID'])

#display table with numeric styling
display(df_AreaSummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_AreaSummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Area_wTotals.csv'))

,AnalysisAreaName,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
AreaID,,,,,,,,,,,,,,,,
0.0,nan,"30,383.7","2,400","4,700","2,300","6,300","16,700","10,400","1,849","3,949",484,"1,789","9,155",0,1,1
1.0,Alpine,"3,803.8",600,600,0,"2,600","4,400","1,800",7,"2,583",24,81,"1,711",1,1,0
3.0,American Fork,"4,265.5","1,100","1,800",700,"8,100","15,800","7,700",126,"7,690",319,"1,015","6,980",2,4,2
4.0,Bluffdale,"3,522.4",100,600,500,"2,300","4,200","1,900",8,"2,256",23,148,"1,803",1,1,0
5.0,Bountiful,"6,180.7","6,900","8,800","1,900","15,500","18,600","3,100",212,"14,908",370,"1,186","2,339",4,4,0
7.0,Cedar Fort,453.1,0,0,0,100,500,400,0,104,4,45,356,0,1,1
8.0,Cedar Hills,"1,035.8",100,100,0,"2,500","2,900",400,1,"2,533",5,6,362,3,3,0
9.0,Centerville,"1,327.6",400,"5,300","4,900","4,800","7,400","2,600",61,"4,618",100,201,"2,514",4,10,6
10.0,Clearfield,"2,128.9","4,300","52,500","48,200","9,000","21,000","12,000",134,"8,040",791,"5,633","7,180",6,35,29


# Class Summary

In [20]:
#Class table is aggregate of df_AASummaryTotals
df_ClassSummaryTable = df_AASummaryTotals.groupby(['ClassOrder','ClassDescription'], as_index=False).agg({"Acres":[np.sum],"acres_redevelopable":[np.sum],"acres_developable":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})

#drop aggregate description from column name (sum, etc)
df_ClassSummaryTable.columns = df_ClassSummaryTable.columns.droplevel(1)

#add totals row
df_ClassSummaryTable_wTotals = df_ClassSummaryTable.append(df_ClassSummaryTable.sum(numeric_only=True).rename('Total'))

#recalculate densities - calculate densities before rounding
df_ClassSummaryTable_wTotals['density_hhemp_orig'  ] = ((df_ClassSummaryTable_wTotals['job_spaces_orig'  ] + df_ClassSummaryTable_wTotals['resunits_orig'     ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_new'   ] = ((df_ClassSummaryTable_wTotals['job_spaces_new'   ] + df_ClassSummaryTable_wTotals['resunits_new'      ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_change'] =  (df_ClassSummaryTable_wTotals['density_hhemp_new'] - df_ClassSummaryTable_wTotals['density_hhemp_orig']).round(0)

#calculate rounded values
df_ClassSummaryTable_wTotals['job_spaces_orig'     ] =  (df_ClassSummaryTable_wTotals['job_spaces_orig'  ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_new'      ] =  (df_ClassSummaryTable_wTotals['job_spaces_new'   ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_change'   ] =   df_ClassSummaryTable_wTotals['job_spaces_new'   ] - df_ClassSummaryTable_wTotals['job_spaces_orig'] 

df_ClassSummaryTable_wTotals['resunits_orig'       ] =  (df_ClassSummaryTable_wTotals['resunits_orig'    ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_new'        ] =  (df_ClassSummaryTable_wTotals['resunits_new'     ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_change'     ] =   df_ClassSummaryTable_wTotals['resunits_new'     ] - df_ClassSummaryTable_wTotals['resunits_orig'] 

#fix last row name to be (HARD CODED CLASS ORDER-FIX!!)
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 666), 'ClassDescription'] = 'Total'
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 666), 'ClassOrder'] = 37

#set index to class order
df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals.set_index(['ClassOrder'])

df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] = df_ClassSummaryTable_wTotals['job_spaces_redeveloped_new'] -  df_ClassSummaryTable_wTotals['job_spaces_redeveloped_orig']
df_ClassSummaryTable_wTotals['resunits_redev_add'         ] = df_ClassSummaryTable_wTotals['resunits_redeveloped_new'  ] -  df_ClassSummaryTable_wTotals['resunits_redeveloped_orig'  ]

df_ClassSummaryTable_wTotals['job_spaces_redev_percentadd'] = df_ClassSummaryTable_wTotals['job_spaces_redev_add'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])
df_ClassSummaryTable_wTotals['job_spaces_dev_percentadd'  ] = df_ClassSummaryTable_wTotals['job_spaces_developed'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])

df_ClassSummaryTable_wTotals['resunits_redev_percentadd'  ] = df_ClassSummaryTable_wTotals['resunits_redev_add'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])
df_ClassSummaryTable_wTotals['resunits_dev_percentadd'    ] = df_ClassSummaryTable_wTotals['resunits_developed'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])

df_ClassSummaryTable_wTotals['Acres_Redev_Percent'        ] =  df_ClassSummaryTable_wTotals['acres_redevelopable'      ]                                                                / df_ClassSummaryTable_wTotals['Acres']
df_ClassSummaryTable_wTotals['Acres_Dev_Percent'          ] =  df_ClassSummaryTable_wTotals['acres_developable'        ]                                                                / df_ClassSummaryTable_wTotals['Acres']

df_ClassSummaryTable_wTotals['Acres_Unchanged_Percent'    ] = 1 - df_ClassSummaryTable_wTotals['Acres_Dev_Percent'] - df_ClassSummaryTable_wTotals['Acres_Redev_Percent']

df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals[['ClassDescription','Acres','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','density_hhemp_orig','density_hhemp_new','density_hhemp_change','Acres_Unchanged_Percent','Acres_Redev_Percent','Acres_Dev_Percent','job_spaces_redev_percentadd','job_spaces_dev_percentadd','resunits_redev_percentadd','resunits_dev_percentadd','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

#display with formatted numbers
display(df_ClassSummaryTable_wTotals.style.format({"Acres":"{:,.0f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","Acres_Unchanged_Percent":"{:,.0%}","Acres_Redev_Percent":"{:,.0%}","Acres_Dev_Percent":"{:,.0%}","job_spaces_redev_percentadd":"{:,.0%}","job_spaces_dev_percentadd":"{:,.0%}","resunits_redev_percentadd":"{:,.0%}","resunits_dev_percentadd":"{:,.0%}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_ClassSummaryTable_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Class_wTotals.csv'))

,ClassDescription,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change,Acres_Unchanged_Percent,Acres_Redev_Percent,Acres_Dev_Percent,job_spaces_redev_percentadd,job_spaces_dev_percentadd,resunits_redev_percentadd,resunits_dev_percentadd,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
ClassOrder,,,,,,,,,,,,,,,,,,,,,,,
1.0,Single Family A1,"18,558",200,200,0,"1,400","14,300","12,900",0,1,1,45%,4%,51%,100%,-0%,6%,94%,95,"1,264",65,830,"12,158"
2.0,Single Family A2,"16,378",500,500,0,"3,200","9,200","6,000",0,1,1,71%,2%,27%,100%,-0%,5%,95%,"1,180","1,970",66,360,"5,680"
3.0,Single Family A3,"18,498","1,500","1,400",-100,"5,600","9,100","3,500",0,1,1,85%,2%,13%,100%,-0%,11%,89%,470,"5,061",53,434,"3,161"
4.0,Single Family A4,"7,297","1,100","1,000",-100,"3,600","7,000","3,400",1,1,0,63%,2%,34%,100%,-0%,5%,95%,23,"3,579",27,190,"3,199"
5.0,Single Family A5,"6,544",900,800,-100,"2,100","4,600","2,500",0,1,1,69%,3%,28%,100%,-0%,6%,94%,97,"1,899",71,224,"2,360"
6.0,Single Family B1,"10,021","1,100","1,100",0,"2,800","15,100","12,300",0,2,2,42%,3%,55%,100%,-0%,4%,96%,70,"2,697",42,558,"11,734"
7.0,Single Family B2,"12,842",800,700,-100,"12,000","23,000","11,000",1,2,1,59%,4%,37%,100%,-0%,8%,92%,54,"11,798",115,"1,052","10,096"
8.0,Single Family B3,"8,649","2,900","2,800",-100,"10,800","16,200","5,400",2,2,0,70%,3%,27%,100%,-0%,8%,92%,39,"10,647",143,545,"4,935"
9.0,Single Family B4,"9,344","1,400","1,300",-100,"10,200","15,600","5,400",1,2,1,71%,3%,25%,100%,-0%,8%,92%,93,"9,896",202,634,"5,016"


In [21]:
#df_AASummaryTotals
df_ClassSummaryTable_wTotals

,ClassDescription,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change,Acres_Unchanged_Percent,Acres_Redev_Percent,Acres_Dev_Percent,job_spaces_redev_percentadd,job_spaces_dev_percentadd,resunits_redev_percentadd,resunits_dev_percentadd,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
ClassOrder,,,,,,,,,,,,,,,,,,,,,,,
1.0,Single Family A1,18557.653528,200.0,200.0,0.0,1400.0,14300.0,12900.0,0.0,1.0,1.0,0.450397,0.035332,0.514272,1.000000,-0.000000,0.059198,0.940802,95.306495,1264.379552,64.982834,830.0,12158.0
2.0,Single Family A2,16378.359826,500.0,500.0,0.0,3200.0,9200.0,6000.0,0.0,1.0,1.0,0.709877,0.017563,0.272560,1.000000,-0.000000,0.049176,0.950824,1179.984403,1970.222352,66.234552,360.0,5680.0
3.0,Single Family A3,18497.521645,1500.0,1400.0,-100.0,5600.0,9100.0,3500.0,0.0,1.0,1.0,0.846823,0.018623,0.134554,1.000000,-0.000000,0.107490,0.892510,469.927619,5061.180120,53.301451,434.0,3161.0
4.0,Single Family A4,7297.470037,1100.0,1000.0,-100.0,3600.0,7000.0,3400.0,1.0,1.0,0.0,0.634263,0.020966,0.344771,1.000000,-0.000000,0.048363,0.951637,22.909249,3578.816766,27.423752,190.0,3199.0
5.0,Single Family A5,6544.004600,900.0,800.0,-100.0,2100.0,4600.0,2500.0,0.0,1.0,1.0,0.689117,0.027284,0.283598,1.000000,-0.000000,0.060894,0.939106,96.890961,1899.067392,70.972310,224.0,2360.0
6.0,Single Family B1,10020.504069,1100.0,1100.0,0.0,2800.0,15100.0,12300.0,0.0,2.0,2.0,0.422104,0.026469,0.551428,1.000000,-0.000000,0.042117,0.957883,70.002654,2697.006901,42.071560,558.0,11734.0
7.0,Single Family B2,12842.008932,800.0,700.0,-100.0,12000.0,23000.0,11000.0,1.0,2.0,1.0,0.590921,0.038812,0.370266,1.000000,-0.000000,0.084930,0.915070,53.504392,11797.793898,114.962931,1052.0,10096.0
8.0,Single Family B3,8649.166278,2900.0,2800.0,-100.0,10800.0,16200.0,5400.0,2.0,2.0,0.0,0.700689,0.030063,0.269248,1.000000,-0.000000,0.075284,0.924716,39.164403,10646.831814,143.223964,545.0,4935.0
9.0,Single Family B4,9344.207920,1400.0,1300.0,-100.0,10200.0,15600.0,5400.0,1.0,2.0,1.0,0.714410,0.032284,0.253306,1.000000,-0.000000,0.079240,0.920760,92.993634,9895.692191,202.329101,634.0,5016.0


# Summary by County

In [22]:
df_CountySummary = df_AASummaryTotals.groupby(['county_id'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum],"resunits_undevelopable":[np.sum],"resunits_remain":[np.sum],"resunits_redeveloped_orig":[np.sum],"resunits_redeveloped_new":[np.sum],"resunits_developed":[np.sum]})
df_CountySummary.columns = df_CountySummary.columns.droplevel(1)

#read in csv with area names
df_CountyNames = pd.read_csv(CountiesTable)

#merge area names with area summary
df_CountySummary_wNames = pd.DataFrame.merge(df_CountyNames, df_CountySummary, on="county_id")

#add total line to area summary with names
df_CountySummary_wNames_wTotals = df_CountySummary_wNames.append(df_CountySummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_CountySummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_CountySummary_wNames_wTotals['job_spaces_orig'  ] + df_CountySummary_wNames_wTotals['resunits_orig'     ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_new'   ] = ((df_CountySummary_wNames_wTotals['job_spaces_new'   ] + df_CountySummary_wNames_wTotals['resunits_new'      ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_change'] =  (df_CountySummary_wNames_wTotals['density_hhemp_new'] - df_CountySummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_CountySummary_wNames_wTotals['job_spaces_orig'     ] = (df_CountySummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_new'      ] = (df_CountySummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_change'   ] =  df_CountySummary_wNames_wTotals['job_spaces_new'    ] - df_CountySummary_wNames_wTotals['job_spaces_orig'] 

df_CountySummary_wNames_wTotals['resunits_orig'       ] = (df_CountySummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_new'        ] = (df_CountySummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_change'     ] =  df_CountySummary_wNames_wTotals['resunits_new'      ] - df_CountySummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_CountySummary_wNames_wTotals = df_CountySummary_wNames_wTotals.set_index(['CO_ORDER'])

#display table with numeric styling
display(df_CountySummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","resunits_undevelopable":"{:,.0f}","resunits_remain":"{:,.0f}","resunits_redeveloped_orig":"{:,.0f}","resunits_redeveloped_new":"{:,.0f}","resunits_developed":"{:,.0f}"}))

#export to csv
df_CountySummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_County_wTotals.csv'))

,county_id,CO_NAME,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,,,,,,
1.0,3.000000,Weber,"60,710.6","39,800","173,500","133,700","86,500","146,100","59,600","2,669","80,447","3,437","11,873","51,183",2,5,3
2.0,1.000000,Davis,"41,697.3","38,900","236,800","197,900","82,300","164,600","82,300",666,"79,594","2,126","21,714","62,689",3,10,7
3.0,2.000000,Salt Lake,"93,878.3","190,300","1,142,700","952,400","356,000","607,200","251,200","3,715","333,659","18,731","114,240","155,751",6,19,13
4.0,4.000000,Utah,"118,991.2","56,600","346,300","289,700","152,900","432,500","279,600","1,981","147,581","3,481","16,800","266,286",2,7,5
10.0,10.000000,nan,"315,277.5","325,700","1,899,200","1,573,500","677,800","1,350,500","672,700","9,031","641,282","27,776","164,627","535,909",3,10,7


In [23]:
df_CountySummary_wNames_wTotals

,county_id,CO_NAME,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,,,,,,
1.0,3.0,Weber,60710.576487,39800.0,173500.0,133700.0,86500.0,146100.0,59600.0,2669.228791,80447.054859,3436.752565,11873.0,51183.0,2.0,5.0,3.0
2.0,1.0,Davis,41697.325948,38900.0,236800.0,197900.0,82300.0,164600.0,82300.0,665.881545,79594.440655,2126.150506,21714.0,62689.0,3.0,10.0,7.0
3.0,2.0,Salt Lake,93878.344964,190300.0,1142700.0,952400.0,356000.0,607200.0,251200.0,3714.681961,333659.019892,18731.299143,114240.0,155751.0,6.0,19.0,13.0
4.0,4.0,Utah,118991.232226,56600.0,346300.0,289700.0,152900.0,432500.0,279600.0,1981.427107,147581.152370,3481.312463,16800.0,266286.0,2.0,7.0,5.0
10.0,10.0,NaN,315277.479625,325700.0,1899200.0,1573500.0,677800.0,1350500.0,672700.0,9031.219405,641281.667775,27775.514676,164627.0,535909.0,3.0,10.0,7.0


# Display HH/Emp Growth by Analysis Areas

In [24]:
df_AnalysisAreas = pd.read_csv(AnalysisAreasTable)

sdf_AnalysisAreas = pd.DataFrame.merge(sdf_AAid, df_AnalysisAreas, on='AreaID')

sdf_AANewHHEmp = pd.DataFrame.merge(sdf_AnalysisAreas, df_AASummaryTotals, on=['AreaID','ClassID'])
sdf_AANewHHEmp

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE,AnalysisAreaName,county_id,ClassDescription,ClassOrder,parcel_id,Acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,72.060894,807.318475,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
1,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,2,2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.0,0.0,0.7,0.7
2,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,72.060894,807.318475,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
3,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,2,2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.0,0.0,0.7,0.7
4,2,254.967270,3.811046e+03,4,a4,"{'rings': [[[420671.3481999999, 4481261.932600...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,72.060894,807.318475,112,53,-59,1362,2470,1108,8.928213,44.949431,58.736514,0.0,0.0,3.995460,1346.229080,11.806696,91.0,1029.0,0.5,0.9,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12373,11135,10436.992442,3.292208e+06,10,h2,"{'rings': [[[412703.1934000002, 4553118.742900...",Clearfield,1.0,Mixed Residential Suburban,35,7331.0,1872.119633,189.356034,189.018404,1701,16808,15107,8424,14739,6315,862.621959,772.635993,65.868814,7593.0,7580.0,129.246837,7530.224340,764.585866,3543.0,3537.0,5.4,16.9,11.5
12374,11136,2229.618596,2.944859e+05,10,g3,"{'rings': [[[414942.4210000001, 4550112.225299...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.721852,102.321081,2563,35690,33127,541,6247,5706,218.365347,1067.335612,1278.049911,12545.0,21860.0,4.990842,509.613234,26.854331,2090.0,3643.0,12.1,163.3,151.2
12375,11137,856.947230,2.329713e+04,10,g3,"{'rings': [[[417057.5292999996, 4550162.428400...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.721852,102.321081,2563,35690,33127,541,6247,5706,218.365347,1067.335612,1278.049911,12545.0,21860.0,4.990842,509.613234,26.854331,2090.0,3643.0,12.1,163.3,151.2
12376,11138,966.598419,5.007495e+04,10,g3,"{'rings': [[[416772.02419999987, 4550546.96120...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.721852,102.321081,2563,35690,33127,541,6247,5706,218.365347,1067.335612,1278.049911,12545.0,21860.0,4.990842,509.613234,26.854331,2090.0,3643.0,12.1,163.3,151.2


In [25]:
##create map centered on Salt Lake
#map_parcels_areas_add = gis.map('Salt Lake')
#
##expression to classify employment growth
#arcade_expression_jobspaceschange = ("var v = $feature.job_spaces_change;"
#                                     "if      (v<0    ) { return 'class0'; }"
#                                     "if      (v<2500 ) { return 'class1'; }"
#                                     "else if (v<5000 ) { return 'class2'; }"
#                                     "else if (v<10000) { return 'class3'; }"
#                                     "else              { return 'class4'; }")
#
##symbology for enrollment classes
#uv_jobspaceschange = [{"value":"class0", "label":"Job Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class1", "label":"0 to 2,500"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class2", "label":"2,500 to 5,000"  , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class3", "label":"5,000 to 10,000" , "symbol":{"type":"esriSFS","color":[149,85,111,168],  "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                      {"value":"class4", "label":"More than 10,000", "symbol":{"type":"esriSFS","color":[168,4,4,168],     "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
#
##define sdf layer
#sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_add,
#                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                            unique_values=uv_jobspaceschange,
#                            arcade_expression=arcade_expression_jobspaceschange,
#                            default_symbol="" #don't include an 'other' category
#                            )
#
##define map characteristics
#map_parcels_areas_add.layout.height='500px'
#map_parcels_areas_add.legend=True
#
##map title
#display(Markdown('<h2><center>Job Growth by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_add

In [26]:
##create map centered on Salt Lake
#map_parcels_areas_addHH = gis.map('Salt Lake')
#
##expression to classify employment growth
#arcade_expression_resunitschange = ("var v = $feature.resunits_change;"
#                                    "if      (v<0   ) { return 'class0'; }"
#                                    "if      (v<1000) { return 'class1'; }"
#                                    "else if (v<2000) { return 'class2'; }"
#                                    "else if (v<5000) { return 'class3'; }"
#                                    "else             { return 'class4'; }")
#
##symbology for enrollment classes
#uv_resunitschange = [{"value":"class0", "label":"HH Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class1", "label":"0 to 1,000"     , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class2", "label":"1,000 to 2,000" , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class3", "label":"2,000 to 5,000" , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#                     {"value":"class4", "label":"More than 5,000", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
#
##define sdf layer
#sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_addHH,
#                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                            unique_values=uv_resunitschange,
#                            arcade_expression=arcade_expression_resunitschange,
#                            default_symbol="" #don't include an 'other' category
#                            )
#
##define map characteristics
#map_parcels_areas_addHH.layout.height='500px'
#map_parcels_areas_addHH.legend=True
#
##map title
#display(Markdown('<h2><center>Household Growth by Analysis Area</center></h2>'))
#
##display map
#map_parcels_areas_addHH

In [27]:
sdf_AA

,FID,city,county,city_lu_ty,gen_lu_typ,max_dua,code,plan_year,plan_sourc,data_sourc,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,599.400903,12203.821403,Bluffdale,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,5673.401678,395913.547830,Bluffdale,a5,"{""rings"": [[[421136.0301000001, 4481104.8302],..."
2,2,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,1554.841774,85616.693206,Bluffdale,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
3,3,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,254.967270,3811.045695,Bluffdale,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
4,4,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,2588.398617,261893.287379,Bluffdale,a5,"{""rings"": [[[421931.24289999995, 4482241.10689..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,11136,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,101.746609,647.000097,Payson,g1,"{""rings"": [[[437436.2742999997, 4432295.444599..."
11137,11137,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,347.071602,6455.345939,Payson,g1,"{""rings"": [[[437517.15270000044, 4432410.1381]..."
11138,11138,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,441.004580,8873.413053,Payson,g1,"{""rings"": [[[437390.2474999996, 4432411.0495],..."
11139,11139,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,743.393043,29068.043955,Payson,g1,"{""rings"": [[[439725.0575000001, 4433219.461200..."
